## Урок 4. Парсинг html, XPath

### Задание 1.

Написать приложение, которое собирает основные новости с сайтов mail.ru, lenta.ru, yandex-новости.

Для парсинга использовать xpath. Структура данных должна содержать:

•	название источника,

•	наименование новости,

•	ссылку на новость,

•	дата публикации


In [36]:
from pprint import pprint
import json
from pymongo import MongoClient
from lxml import html
import requests

header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) YaBrowser/20.2.0.1145'}

# список масок XPath дла поиска новостей по сайтам
mask  = {'yandex': { 'site': "https://yandex.ru/news/",
                     'tag' : "//div[@class='story__topic']",
                     'name': ".//h2[@class='story__title']/a/text()",
                     'source': "..//div[@class='story__info']/div[@class='story__date']/text()",
                     'date': "..//div[@class='story__info']/div[@class='story__date']/text()",
                     'href': ".//h2[@class='story__title']/a/@href"},

         'lenta' : { 'site': 'https://lenta.ru/',
                     'tag': "//div[@class='span4']//div[@class='item']//a",
#                     'tag': "//div[@class='span4']//div[@class='item']//a|//div[@class='span4']//div[@class='first-item']//a|//div[@class='row']//a//span",
                     'name': "./text()",
                     'source': ".//@href",
                     'date': ".//@datetime",
                     'href': ".//@href"},
                   
         'mail'  : { 'site': 'https://news.mail.ru/',
                     'tag': "//div[@class='block']//li//a",
                     'name': "./text()",
                     'source': "./text()",
                     'date': "./text()",
                     'href': ".//@href"}}


# вернуть список новостей с сайта site_name
def writenews_to_db(site_name):

    try:
        # скачать с сайта site_name все информационные теги по маске tag    
        response = requests.get(mask[site_name]['site'], headers = header)
        root = html.fromstring(response.text)
        news_list = root.xpath(mask[site_name]['tag'])
        print('Всего на сайте новостей:' , len(news_list))
        
        # по каждой найденной новости в списке разобрать значения   
        for item in news_list:
            news_item = {}
            news_item['site']    = mask[site_name]['site']
            news_item['name']    = item.xpath(mask[site_name]['name'])[0]
            news_item['source']  = item.xpath(mask[site_name]['source'])[0]
            news_item['date']    = item.xpath(mask[site_name]['date'])[0]
            news_item['href']    =  mask[site_name]['site'] + item.xpath(mask[site_name]['href'])[0]
            # специальная дообработка особенностей Яндекса
            if site_name == 'yandex':
                news_item['source'] = news_item['source'][:-5].replace('вчера','').replace('\xA0в','')
                news_item['date']   = news_item['date'][-5:]
            # специальная дообработка особенностей Мail
            if site_name == 'mail':
                news_item['source'] = "неизвестно"
                news_item['date']   = "неизвестно"
           # специальная дообработка особенностей Ленты
            if site_name == 'lenta':
                news_item['source'] = 'www.lenta.ru'
            # распечатать разобранную новость
            print(news_item['site'])
            print(news_item['name'])
            print(news_item['source'])
            print(news_item['date'])
            print(news_item['href'])

            # записать в базу данных новую уникальную новость
            if news.count_documents(news_item) == 0:
                news.insert_one(news_item)
#                print('В базу данных добавлена новая строка с новостной информацией с сайта: ', news_item['site'])
#                print(news_item)
            print()

        return 
    except:
        print('Ошибка запроса')
            

# создаем базу данных "news" в MongoDB
client  = MongoClient('localhost',27017)
db = client['news']
news = db.news
            
# распарсить, распечатать и сохранить новости Яндекса
writenews_to_db('yandex')
writenews_to_db('mail')
writenews_to_db('lenta')
   
    
    
    
    
    

Всего на сайте новостей: 65
https://yandex.ru/news/
Россия временно остановит въезд граждан Китая
RT на русском 
00:23
https://yandex.ru/news//news/story/Rossiya_vremenno_ostanovit_vezd_grazhdan_Kitaya--17cac59a3fa5dded1581fb2bf19aeb4d?lr=213&lang=ru&stid=swXTVIn1KacgZoDEGbKT&persistent_id=88572567&rubric=index&from=index

https://yandex.ru/news/
Сурков освобожден от должности помощника Путина
Газета.Ru  
23:32
https://yandex.ru/news//news/story/Surkov_osvobozhden_ot_dolzhnosti_pomoshhnika_Putina--2aacec516e4013d1e07231139f57efbc?lr=213&lang=ru&stid=96YzJGS7TuJE2Bq3MOpQ&persistent_id=88595175&rubric=index&from=index

https://yandex.ru/news/
Бывшим акционерам ЮКОСа снова присудили компенсацию в $50 млрд
BFM.ru  
23:36
https://yandex.ru/news//news/story/Byvshim_akcioneram_YUKOSa_snova_prisudili_kompensaciyu_v_$50_mlrd--6d51a61f62bfa3c5e5bdf818bf9c9ffb?lr=213&lang=ru&stid=AWe16jw2xs37oZyzRuhd&persistent_id=88527570&rubric=index&from=index

https://yandex.ru/news/
В Конституции могут закре

Всего на сайте новостей: 8
https://news.mail.ru/
ФСБ задержала подростков, готовивших теракты в Крыму
неизвестно
неизвестно
https://news.mail.ru//incident/40634942/

https://news.mail.ru/
Как выглядит оружие римского легионера до и после очистки (фото)
неизвестно
неизвестно
https://news.mail.ru//society/40635200/

https://news.mail.ru/
Продавщица из США придумала необычную рекламу магазину (фото)
неизвестно
неизвестно
https://news.mail.ru//society/40634150/

https://news.mail.ru/
Этери Тутберидзе объяснила, почему не хочет общаться со СМИ
неизвестно
неизвестно
https://news.mail.ru/https://sportmail.ru/news/figure-skating/40636859/

https://news.mail.ru/
Принят закон о бесплатном горячем питании в начальных классах
неизвестно
неизвестно
https://news.mail.ru//politics/40631052/

https://news.mail.ru/
Женщина случайно нашла аудиокассету, потерянную 25 лет назад
неизвестно
неизвестно
https://news.mail.ru//society/40630061/

https://news.mail.ru/
В Волгограде засняли окутанную туманом «Роди